In [71]:
from glob import glob
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

# x=[]
# y=[]
# for fname in tqdm (glob("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC/train/*.jpg")):
#     img = cv.imread(fname)
#     print(fname)
#     #     txt_fname = fname.replace("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC/img", 
#     #                               "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC/txt")
#     txt_fname = txt_fname.replace(".jpg", ".txt")
    
#     # DF reader
#     txt_df=pd.read_csv(txt_fname, 
#                 names=['x1', 'y1', 'x2', 'y2', 'class'],  
#                 dtype={'x1': np.int, 'y1': np.int, 'x2': np.int, 'y2': np.int, 'class':str} )
    
#     h, w, c = img.shape
#     x.append(w) 
#     y.append(h)
#     break
    
    
# # sns.set_style("darkgrid")
# # sns.jointplot(x, y, kind="kde")

  0%|                                                                                                                                                                                                                                | 0/8930 [00:00<?, ?it/s]

D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC/train\4609080-H-TOP-NEW-0626_22_0_1-u6___A0.jpg


In [2]:
from glob import glob
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import os

#### Helper Function
def createDirectory(path):
    if not os.path.isdir(path):
        os.makedirs(path)
        print ("Creating {}".format(path))
    return path


In [3]:

def generate_multiline_images_from_singleline():
    
    #Variable
    src_img_dir = "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Multiline/raw/*.jpg"
    out_dir = createDirectory("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Multiline/multiline")
    fixed_width = 412
    overlap_limit = 100

    #Member function
    def generate_multiline(
        img,
        fixed_width = 412,
        overlap_limit = 100,
        out_txt_file='test.txt',
        out_img_file='test.jpg'
    ):
        #Image size
        W = img.shape[1]
        H = img.shape[0]
        if (W <= fixed_width):
            row_count = 1
        else:
            row_count = W // fixed_width +1
            if (W%fixed_width < overlap_limit):
                row_count =row_count-1
        #print("row_count: ",row_count)    

        #Image generate
        blank_image = np.zeros(shape=[H * row_count, fixed_width+overlap_limit, 3], dtype=np.uint8)
        new_label = []
        color = (0, 0, 255)

        # Do Multiline
        for i in range(row_count):
            start = i * fixed_width
            stop  = start + fixed_width + overlap_limit
            stop = W if  (stop > W) else stop
            #print("start: ", start, "stop: ", stop)

            temp = img.copy()
            for j, r in txt_df.iterrows():
                if (int(r['x2'])< stop and int(r['x1'])< stop and int(r['x1'])> start and int(r['x2'])> start ):
                    new_r = r.to_dict()

                    new_r['x1'] = new_r['x1'] - i*fixed_width
                    new_r['x2'] = new_r['x2'] - i*fixed_width

                    new_r['y1'] = new_r['y1'] + i*H
                    new_r['y2'] = new_r['y2'] + i*H
                    new_label.append(new_r)
                    #cv.rectangle(img, (r['x1'],r['y1']), (r['x2'],r['y2']), color, 1)
            blank_image[i*H:(i+1)*H, :stop-start] = temp[:, start:stop]
            
        #Pad
        top = int( (fixed_width + overlap_limit - blank_image.shape[0]) /2) 
        top = top if top >0 else 0
        bottom = top
        left = 0
        right = left
        #print(name, blank_image.shape)
        blank_image = cv.copyMakeBorder(blank_image, top, bottom, left, right, borderType=cv.BORDER_CONSTANT, value=[0,0,0])


        # Output
        with open(out_txt_file,"w") as f:
            for r in new_label:
                r['y1'] = r['y1'] + top
                r['y2'] = r['y2'] + top
                f.write("{},{},{},{},{}\n".format(r['x1'],r['y1'], r['x2'],r['y2'], r['class']))
                #cv.rectangle(blank_image, (r['x1'],r['y1']), (r['x2'],r['y2']), color, 1)
            #cv.imshow("Images", blank_image)
            #cv.waitKey(0)
            cv.imwrite(out_img_file, blank_image)

    '''
    MAIN
    '''
    for fname in tqdm (glob(src_img_dir)):
        #Image
        img = cv.imread(fname)
        name = os.path.basename(fname)

        # DF reader
        txt_fname = fname.replace(".jpg", ".txt")    
        txt_df=pd.read_csv(txt_fname, 
                    names=['x1', 'y1', 'x2', 'y2', 'class'],  
                    dtype={'x1': np.int, 'y1': np.int, 'x2': np.int, 'y2': np.int, 'class':str} )

        #Settings
        out_txt_file= os.path.join(out_dir, name.replace(".jpg", ".txt"))
        out_img_file= os.path.join(out_dir, name)

        generate_multiline(    
            img,
            fixed_width,
            overlap_limit,
            out_txt_file,
            out_img_file) 
    


In [4]:
if 1:
    generate_multiline_images_from_singleline()


  0%|                                                                                                                                                                                                                                | 0/9112 [00:00<?, ?it/s]

Creating D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinOnly/QFP+SOIC-Multiline/multiline


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9112/9112 [02:32<00:00, 59.87it/s]
